# Check fo duplicate articles and data objects
A list of publications is obtainded from the app database. This list will contain a titles, IDs and DOIs which need to be explored to look for duplicates. 
The steps of the process are: 
 1. get a Title, DOI, and URL for each publication
 2. revise each element of the list for duplicates

In [12]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
from tqdm import tqdm
#library for handling json files
import json
# library for using regular expressions
import re
# library for handling http requests
import requests
# import custom functions (common to various notebooks)
import processing_functions as pr_fns

current_step = 2

## Verify if there are duplicates in articles

1. Open the current publication list from the appdb
2. read each entry and check if there are duplicates in doi, url or title



In [13]:
# 1 current app DB

ukchapp_db = "db_files/app_db20211005.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()

#  get publication data from the ukch app
app_pubs = pr_fns.get_pub_data(ukchapp_db)

# 2 read each entry and check if there are duplicates in doi, url or title
dup_list={}
dup_count = 0
if current_step == 1:  
    dups = []
    for idx, a_pub in enumerate(tqdm_notebook(app_pubs)):
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        # verfy if dois are duplicated
        if pub_doi != None:
            for i_indx in range(idx+1, len(app_pubs)):
                #print(pub_doi, app_pubs[i_indx][2])
                if app_pubs[i_indx][2]!=None and pub_doi.strip().lower() ==  app_pubs[i_indx][2].strip().lower():
                    print("DOI", pub_doi, "duplicated at:", i_indx, app_pubs[i_indx], app_pubs[idx] )
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'DOI'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if urls are all unique
        if pub_url != None:
            for i_indx in range(idx+1, len(app_pubs)):
                if app_pubs[i_indx][3]!=None and pub_url.strip().lower() ==  app_pubs[i_indx][3].strip().lower():
                    print("URL", pub_url, "duplicated at:", i_indx, app_pubs[i_indx], app_pubs[idx] )    
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'URL'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if titles are all unique
        if pub_title != None:
            for i_indx in range(idx+1, len(app_pubs)):
                similarity = txtc.similar(pub_title.strip().lower(), app_pubs[i_indx][1].strip().lower())
                #print(similarity)
                if app_pubs[i_indx][1]!=None and pub_title.strip().lower() ==  app_pubs[i_indx][1].strip().lower():
                    print("Title", pub_title, "duplicated at:", i_indx,app_pubs[i_indx][1], app_pubs[idx][1])     
                    print("Similarity:", similarity)
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'Title'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
                elif similarity > 0.8:
                    print(similarity, "Title:\n\t", pub_title, "\n\t- similar at:\n\t", i_indx,app_pubs[i_indx][1]) 
if len(dup_list) > 0:
        csvh.write_csv_data(dup_list, 'db_duplicates202111.csv')

## Verify if there are duplicates in data objects

1. get the current data objects list from the appdb
2. read each entry and check if there are duplicates in doi, url or title



In [23]:
# 1 current app DB

ukchapp_db = "db_files/app_db20211005.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()

#  get datasets list from the ukch app
app_datasetes = pr_fns.get_dataset_data(ukchapp_db)

# 2 read each entry and check if there are duplicates in doi, url or title
if current_step == 2:  
    dup_list={}
    dup_count = 0
    for idx, a_ds in enumerate(tqdm_notebook(app_datasetes)):
        ds_id = a_ds[0]
        ds_doi = a_ds[1]
        ds_url = a_ds[2]
        ds_name = a_ds[3]
        #print (ds_id, ds_doi, ds_url, ds_name)
        # verfy if dois are duplicated
        if ds_doi != None and ds_doi != '':
            for i_indx in range(idx+1, len(app_datasetes)):
                #print(pub_doi, app_pubs[i_indx][1])
                if app_datasetes[i_indx][1]!=None and ds_doi.strip().lower() ==  app_datasetes[i_indx][1].strip().lower():
                    print("DOI", ds_doi, "duplicated at:", i_indx, app_datasetes[i_indx], app_datasetes[idx] )
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'DOI'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if urls are all unique
        if ds_url != None:
            for i_indx in range(idx+1, len(app_datasetes)):
                #print(app_datasetes[i_indx])
                if app_datasetes[i_indx][2]!=None and ds_url.strip().lower() ==  app_datasetes[i_indx][2].strip().lower():
                    print("URL", ds_url, "duplicated at:", i_indx, app_datasetes[i_indx], app_datasetes[idx] )  
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'URL'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if titles are all unique
        if ds_name != None:
            for i_indx in range(idx+1, len(app_datasetes)):
                similarity = txtc.similar(ds_name.strip().lower(), app_datasetes[i_indx][3].strip().lower())
                #print(similarity)
                if app_datasetes[i_indx][1]!=None and ds_name.strip().lower() ==  app_datasetes[i_indx][3].strip().lower():
                    print("Title", ds_name, "duplicated at:", i_indx,app_datasetes[i_indx][3], app_datasetes[idx][3])     
                    print("Similarity:", similarity)
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'Title'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
                elif similarity > 0.8:
                    print(similarity, "Title:\n\t", ds_name, "\n\t- similar at:\n\t", i_indx,app_datasetes[i_indx][3]) 
                    break
if len(dup_list) > 0:
        csvh.write_csv_data(dup_list, 'db_duplicate_do_202111.csv')

<ipython-input-23-d4bbfa980129>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, a_ds in enumerate(tqdm_notebook(app_datasetes)):


  0%|          | 0/442 [00:00<?, ?it/s]

Title The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5 duplicated at: 3 The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5 The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5
Similarity: 1.0
0.9818181818181818 Title:
	 The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5 
	- similar at:
	 4 The application of QENS to investigate the methanol-to-gasoline reaction over ZSM-5
0.9818181818181818 Title:
	 The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5 
	- similar at:
	 4 The application of QENS to investigate the methanol-to-gasoline reaction over ZSM-5
0.896551724137931 Title:
	 41929_2019_334_MOESM1_ESM.pdf 
	- similar at:
	 27 41929_2018_213_MOESM1_ESM.pdf
0.85 Title:
	 cs9b00685_si_001.pdf 
	- similar at:
	 19 cs8b00389_si_001.pdf
0.8372093023255814 Title:
	 cctc201901268-sup-0001-misc_information.pdf 
	- similar at:
	 12 chem

0.8666666666666667 Title:
	 c5cc08714g1.pdf 
	- similar at:
	 145 c5cc08681g1.pdf
0.8837209302325582 Title:
	 cssc201501225-sup-0001-misc_information.pdf 
	- similar at:
	 172 cctc201600925-sup-0001-misc_information.pdf
0.85 Title:
	 ja5b09913_si_001.pdf 
	- similar at:
	 148 ja5b09913_si_002.cif
0.9 Title:
	 cs502038y_si_001.pdf 
	- similar at:
	 150 ic5b02038_si_001.pdf
0.85 Title:
	 ic5b02038_si_001.pdf 
	- similar at:
	 151 ic5b02038_si_003.cif
0.85 Title:
	 ic5b02038_si_003.cif 
	- similar at:
	 155 ic5b02233_si_002.cif
0.85 Title:
	 ma5b01293_si_001.pdf 
	- similar at:
	 156 ma5b00225_si_001.pdf
0.85 Title:
	 ic5b02233_si_001.pdf 
	- similar at:
	 155 ic5b02233_si_002.cif
0.85 Title:
	 ma5b00225_si_001.pdf 
	- similar at:
	 193 cs5b00625_si_001.pdf
0.85 Title:
	 cs5b01327_si_001.pdf 
	- similar at:
	 193 cs5b00625_si_001.pdf
0.85 Title:
	 om501252m_si_001.pdf 
	- similar at:
	 159 om501252m_si_002.cif
0.85 Title:
	 ja5062467_si_001.pdf 
	- similar at:
	 162 ja5062467_si_002.cif
0

	 315 1-s2.0-S0926337318306167-mmc1.docx
0.9117647058823529 Title:
	 1-s2.0-S0926337318306167-mmc1.docx 
	- similar at:
	 316 1-s2.0-S0926337318307136-mmc1.docx
0.8529411764705882 Title:
	 1-s2.0-S0926337318307136-mmc1.docx 
	- similar at:
	 317 1-s2.0-S092633731930400X-mmc1.docx
0.8823529411764706 Title:
	 1-s2.0-S0013468617312628-mmc1.docx 
	- similar at:
	 323 1-s2.0-S0013468618315172-mmc1.docx
0.9117647058823529 Title:
	 1-s2.0-S0360319916304268-mmc1.docx 
	- similar at:
	 326 1-s2.0-S0360319916324387-mmc1.docx
0.8656716417910447 Title:
	 1-s2.0-S0360319916324387-mmc1.docx 
	- similar at:
	 327 1-s2.0-S0360319916303135-mmc1.doc
0.8059701492537313 Title:
	 1-s2.0-S0360319916303135-mmc1.doc 
	- similar at:
	 335 1-s2.0-S0039602816301509-mmc1.docx
0.8529411764705882 Title:
	 1-s2.0-S0021951714000876-mmc1.docx 
	- similar at:
	 329 1-s2.0-S0021951716001135-mmc1.docx
0.8529411764705882 Title:
	 1-s2.0-S0021951716001135-mmc1.docx 
	- similar at:
	 330 1-s2.0-S0021951718302124-mmc1.docx
0

In [22]:
dup_list

{1: {'pub_comp': (3,
   '10.5286/ISIS.E.79110599',
   'https://data.isis.stfc.ac.uk/doi/INVESTIGATION/79110599/',
   'The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5'),
  'pub_dup': (4,
   '10.5286/ISIS.E.82355314',
   'https://data.isis.stfc.ac.uk/doi/INVESTIGATION/82355314/',
   'The application of INS to investigate the methanol-to-gasoline reaction over ZSM-5'),
  'dup_at': 'Title'},
 2: {'pub_comp': (464,
   '10.5286/ISIS.E.86778394',
   'https://data.isis.stfc.ac.uk/doi/STUDY/103200776/',
   'Investigating the speciation of hydrocarbonaceous deposits formed during oxidative dehydrogenation of propane'),
  'pub_dup': (465,
   '10.5286/ISIS.E.90681467',
   'https://data.isis.stfc.ac.uk/doi/STUDY/103195237/',
   'Investigating the speciation of hydrocarbonaceous deposits formed during oxidative dehydrogenation of propane'),
  'dup_at': 'Title'},
 3: {'pub_comp': (513,
   '10.5517/ccdc.csd.cc25wv1z',
   'https://www.ccdc.cam.ac.uk/structures/Search?

## Check that pdf files exist 

Use the data on the articles table to verify if file are stored in the corresponding folder
We also check that the files in the folder are all accounted for (have a corersponding record)

In [ ]:
if current_step == 2:
    # get publication data from the ukch app
    app_pubs = pr_fns.get_pub_data(ukchapp_db)

    dups = []
    for idx, a_pub in ennumerate(tqdm_notebook(app_pubs)):
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        for i_indx in range(idx,len(app_pubs)):
            if pub_doi.strip().lower() ==  app_pubs[i_idx][2]:
                print(pub_doi, "duplicated at:" i_indx) 
                               



In [ ]:
if current_step == 2:
    for infile in tqdm_notebook(Path("pdf_files").glob('*.pdf')):
        file_found = False
        for a_pub in app_pubs:
            if infile.name == a_pub[4]:
                file_found = True
                break
        if not file_found:
            print("Not in DB:", infile.name)

## Get missing pdfs

In [ ]:
# use regular expression to check if a given string
# is a valid DOI, using pattern from CR
def valid_doi(cr_doi):
    # CR DOIS: https://www.crossref.org/blog/dois-and-matching-regular-expressions/
    # CR DOIs re1
    # /^10.\d{4,9}/[-._;()/:A-Z0-9]+$/i
    if cr_doi == None:
        return False
    cr_re_01 = '^10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.match(cr_re_01, cr_doi, re.IGNORECASE)
    if compare != None and cr_doi == compare.group():
        return True
    else:
        return False
# get publication data from the ukch app
db_pubs = pr_fns.get_pub_data(ukchapp_db)

if current_step == 2:
    for a_pub in tqdm_notebook(db_pubs):
        if a_pub[0] > 616:
            pub_id = a_pub[0]
            pub_title = a_pub[1]
            pub_doi = a_pub[2]
            pub_url = a_pub[3]
            pub_pdf = a_pub[4]
            pub_html = a_pub[5]
            if pub_pdf == None:
                not_in_url = True
                print("ID: ", pub_id, "Publication: ",pub_title,
                      "\n\tDOI: ", pub_doi, " URL: ", pub_url)
                if "pdf" in pub_url:
                    print ("\tTry to get the pdf from URL: ", pub_url)
                    try:
                        response = requests.get(pub_url)
                        content_type = response.headers['content-type']
                        if not 'text' in content_type:
                            #print(response.headers)
                            cd= response.headers['content-disposition']
                            #print(cd)
                            fname = re.findall("filename=(.+)", cd)[0]
                            #print(fname)
                            if not Path('pdf_files/' + pdf_file).is_file():
                                with open('pdf_files/'+ fname +'.pdf', 'wb') as f:
                                    f.write(response.content)
                            else:
                                set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            not_in_url = False
                    except:
                        print("ID: ", pub_id, "\nPublication: ",pub_title, 
                               "\nDOI: ", pub_doi, "\nDOI: ", pub_url) 
                if not_in_url:
                    print("\tTry to see if json file has link to pdf: ")
                    if valid_doi(pub_doi):
                        crjd, doi_file = pr_fns.get_cr_json_object(pub_doi)
                        got_pdf = False
                        if "link" in crjd.keys():
                            for a_link in crjd["link"]:
                                if "\tURL" in a_link.keys() and ("pdf" in a_link["URL"] or "pdf" in a_link["content-type"]):
                                    cr_url = a_link["URL"]
                                    #print("URL: ", cr_url)
                                    pdf_file = get_pdf_from_url(cr_url)
                                    # if the name corresponds to a existing file, assign value to db_record
                                    if Path('pdf_files/' + pdf_file).is_file():
                                        print("\tFile name:", pdf_file)
                                        set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                                        got_pdf = True
                                    else:
                                        print("\tcould not get file from", cr_url)
                        else: 
                            print("\tno links in json", pub_doi)
                    if not got_pdf and "elsevier" in pub_url:
                        print("\tTrying elsevier doi:" )
                        pdf_file = pr_fns.get_elsevier_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    elif not got_pdf and "wiley" in pub_url:
                        print("\tTrying wiley doi:" )
                        pdf_file = pr_fns.get_wiley_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    elif not got_pdf and "pubs.acs" in pub_url:
                        print("\tTrying acs doi:" )
                        pdf_file = pr_fns.get_acs_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    if not got_pdf:
                        print("\tTry doi:  https://doi.org/" + pub_doi)
    


## Use pdfminer to get metadata from pdf file

In [ ]:
import pdfminer
from pdfminer import high_level as pdfmnr_hl

# functions for PDFminer

def get_pdf_text(pdf_file):
    return pdfmnr_hl.extract_text(pdf_file)

# get the paragraph fragments with references to data
def get_ref_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if pr_fns.is_data_stmt(sentence.lower()):
            idx = sentences.index(sentence)
            groups.append([idx-1,idx,idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        return_group.append(full_sentence)
    return return_group

# get the paragraph fragments with references to data
def get_all_data_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if 'data' in sentence.lower() or 'inform' in sentence.lower():
            idx = sentences.index(sentence)
            groups.append([idx-1, idx, idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        if not full_sentence in return_group:
            return_group.append(full_sentence)
    return return_group

# get the http strings from references to data
def get_http_ref(sentence):
    http_frag = ""
    if 'http' in sentence.lower():
        idx_http = sentence.lower().index('http')
        http_frag = sentence[idx_http:]
        space_in_ref = True
        while " " in http_frag:
            space_idx = http_frag.rfind(" ")
            http_frag = http_frag[:space_idx]
        if(http_frag[-1:]=="."):
            http_frag = http_frag[:-1]
    return http_frag

In [ ]:
if current_step == 2:
    # get publication data from the ukch app
    db_pubs = pr_fns.get_pub_data(ukchapp_db)

    # get the list of dois already mined for data 
    input_file = 'pub_data_add202012.csv'
    id_field = 'num'
    processed, headings = csvh.get_csv_data(input_file, id_field)
    for id_num in processed:
        current_title = processed[id_num]['doi']
    processed[1]['num']

    processed_dois = []
    for entry in processed:
        if not processed[entry]['doi'] in processed_dois:
            processed_dois.append( processed[entry]['doi'])

    data_records = {}
    data_mentions = {}
    ref_count = mention_count = 0
    for a_pub in tqdm_notebook(db_pubs):
        data_refs = []
        data_sents = []
        if a_pub[0] > 616:
            pub_id = a_pub[0]
            pub_title = a_pub[1]
            pub_doi = a_pub[2]
            pub_url = a_pub[3]
            pub_pdf = a_pub[4]
            pub_html = a_pub[5]
            if pub_pdf == 'None':
                print("*************************")
                print("Missing PDF for:", pub_doi)
                print("*************************")
            else:
                pdf_file = "pdf_files/" + pub_pdf
                if not Path(pdf_file).is_file():
                    print("*************************")
                    print("Missing file for:", pdf_file, "for", pub_doi)
                    print("*************************")
                else: 
                    print("PDF filename", pdf_file)
                    pdf_text = get_pdf_text(pdf_file)
                    ref_sentences = get_ref_sentences(pdf_text)
                    data_sentences = get_all_data_sentences(pdf_text)
                    for r_sentence in ref_sentences:
                        dt_link = get_http_ref(r_sentence)
                        if 'supplem' in r_sentence.lower():
                            data_refs.append({'type':'supplementary',"desc":r_sentence, 'data_url':dt_link})
                        else:
                            data_refs.append({'type':'supporting',"desc":r_sentence, 'data_url':dt_link})
                    for d_sentence in data_sentences:
                        dt_link = get_http_ref(d_sentence)
                        if 'supplem' in d_sentence.lower():
                            data_sents.append({'type':'supplementary',"desc":d_sentence, 'data_url':dt_link})
                        else:
                            data_sents.append({'type':'supporting',"desc":d_sentence, 'data_url':dt_link})
            if data_refs != []:
                for data_ref in data_refs:
                    data_record = {'id':pub_id, 'doi':pub_doi}    
                    data_record.update(data_ref)
                    data_records[ref_count] = data_record
                    ref_count += 1
            if data_sents != []:
                for data_sent in data_sents:
                    sentence_record = {'id':pub_id, 'doi':pub_doi}    
                    sentence_record.update(data_sent)
                    data_mentions[mention_count] = sentence_record
                    mention_count += 1

Write the results to a csv file to be checked to verify if data mentions are actually 

In [ ]:
#if len(data_records) > 0:
#    csvh.write_csv_data(data_records, 'pdf_data.csv')
if current_step == 2:    
    if len(data_mentions) > 0:
        csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')

Verify if the mentions of data or information actually can be linked to data objects

In [ ]:
from IPython.display import clear_output

if current_step == 3:
    print(ukchapp_db)
    print(len(app_pubs))
    # Open results file
    data_mentions, dm_headers = csvh.get_csv_data('pdf_mentions202110.csv')
    print(dm_headers)
    art_id = ''
    for dm in data_mentions:
        if data_mentions[dm]['action']=='':
            clear_output()
            print ("*******************************************")
            print ("Article id  :", data_mentions[dm]['id'])
            print ("DOI         :", data_mentions[dm]['doi'])
            print ("Type        :", data_mentions[dm]['type'], '\tLine:', dm)
            print ("Description :\n\t", data_mentions[dm]['desc'])
            print ("data_url :", data_mentions[dm]['data_url'])
            print ("*******************************************")
            decide_action = False
            while not decide_action:
                print('Action:')
                print('\ta) review')
                print('\tb) none')
                print('\tSelect a or b:')
                lts = input()
                if lts == "a":
                    data_mentions[dm]['action'] = 'review'
                    decide_action = True
                elif lts == "b":
                    data_mentions[dm]['action'] = 'none'
                    decide_action = True
        art__id = data_mentions[dm]['id']
        if dm > 1700:
            break
    if len(data_mentions) > 0:
       csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')
    

In [ ]:
# clear the output after each loop cycle
from IPython.display import clear_output

# display editable spreadsheet
import ipysheet


# show gds parameters in a spreadsheet on jupyter
def show_gds(gds_group):
    gds_list = gds_to_list(gds_group)
    #print(gds_list)
    #add 10 more rows in case we need more parameters
    for i in range(10):
        gds_list.append([(len(gds_list)-1)+1,None,None,None,None])
    a_sheet = ipysheet.sheet(rows=len(gds_list), columns=len(gds_list[0]))
    ipysheet.cell_range(gds_list)
    display(a_sheet)
    return a_sheet


if current_step == 3:
    print(ukchapp_db)
    print(len(app_pubs))
    # Open results file
    data_mentions, dm_headers = csvh.get_csv_data('pdf_mentions202110.csv')
    print(dm_headers)
    art_id = ''
    terminate = False
    additional_rows = {}
    for dm in data_mentions:
        if data_mentions[dm]['action']=='review':
            clear_output()
            print ("*******************************************")
            print ("Article id  :", data_mentions[dm]['id'])
            print ("DOI         :", data_mentions[dm]['doi'])
            print ("Type        :", data_mentions[dm]['type'], '\tLine:', dm)
            print ("Description :\n\t", data_mentions[dm]['desc'])
            print ("data_url :", data_mentions[dm]['data_url'])
            print ("*******************************************")
            decide_action = False
            while not decide_action:
                print('Action:')
                print('\ta) review: https://doi.org/'+data_mentions[dm]['doi'])
                print('\ts) add new row')
                print('\td) next')
                print('\tf) terminate')
                print('\tSelect a, s, d, f:')
                lts = input()
                if lts == "a":
                    data_mentions[dm]['action'] = 'reviewed'
                    print ('https://doi.org/'+data_mentions[dm]['doi'])
                    print ('link:',data_mentions[dm]['link'])
                    add_this = input()
                    data_mentions[dm]['link'] = add_this
                    print ('issue:',data_mentions[dm]['issue'])
                    add_this = input()
                    data_mentions[dm]['issue'] = add_this
                    print ('name:',data_mentions[dm]['name'])
                    add_this = input()
                    data_mentions[dm]['name'] = add_this
                    print ('file:',data_mentions[dm]['file'])
                    add_this = input()
                    data_mentions[dm]['file'] = add_this
                if lts == "s":
                    #add a new row
                    new_idx = len(data_mentions) + len(additional_rows) + 1
                    additional_rows[new_idx] = data_mentions[dm]
                    print ('link:',additional_rows[new_idx]['link'])
                    add_this = input()
                    additional_rows[new_idx]['link'] = add_this
                    print ('issue:',additional_rows[new_idx]['issue'])
                    add_this = input()
                    additional_rows[new_idx]['issue'] = add_this
                    print ('name:',additional_rows[new_idx]['name'])
                    add_this = input()
                    additional_rows[new_idx]['name'] = add_this
                    print ('file:',additional_rows[new_idx]['file'])
                    add_this = input()
                    additional_rows[new_idx]['file'] = add_this
                elif lts == "d":
                    if data_mentions[dm]['action'] != 'reviewed':
                        data_mentions[dm]['action'] = 'none'
                    decide_action = True
                elif lts == 'f':
                    decide_action = True
                    terminate = True
        art__id = data_mentions[dm]['id']
        if dm > 1700 or terminate:
            break
    if len(additional_rows)> 0 :
        for nr in additional_rows:
            for a_field in dm_headers:
                data_mentions[nr][a_field] = additional_rows[nr][a_field]
    if len(data_mentions) > 0:
       csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')

In [ ]:
filter_mentions = {}
for dm in data_mentions:
    if data_mentions[dm]['action'] in ['add', 'reviewed']:
        filter_mentions[dm]={}
        for a_field in dm_headers:
            filter_mentions[dm][a_field] = data_mentions[dm][a_field]
print('filtered mentions:', len(filter_mentions))

In [ ]:
new_do_id_list =[]
for fm in filter_mentions:
    art_id = int(filter_mentions[fm]["id"])
    if not art_id in new_do_id_list:
        new_do_id_list.append(art_id)

# currend app DB
ukchapp_db = "db_files/app_db20211005.sqlite3"

no_data_pubs = pr_fns.get_pub_app_no_data(ukchapp_db)
#print(len(ids_w_data))
print(len(no_data_pubs))
print(new_do_id_list, len(new_do_id_list))
filter_mentions


int_idx = 0
revised_list = {}
if Path("./html_revised202111.csv").is_file():
    revised_list, rl_headers = csvh.get_csv_data('html_revised202111.csv')
    int_idx = len(revised_list)
    
already_revised =[]
for fm in revised_list:
    art_id = int(revised_list[fm]["id"])
    if not art_id in already_revised:
        already_revised.append(art_id)
    
for ndp in no_data_pubs:
    if not ndp[0] in new_do_id_list and ndp[0] > 616 and not ndp[0] in already_revised:
        int_idx += 1
        pub_id = ndp[0]
        pub_title = ndp[1]
        pub_doi = ndp[2]
        pub_url = ndp[3]
        data_record = {'id':pub_id, 'doi':pub_doi, 'title':pub_title} 
        print ('id',pub_id, '\n', pub_title)
        decide_action = False
        terminate = False
        while not decide_action:
            print('Action:')
            print(pub_url)
            print("https://doi.org/"+pub_doi)
            print('\ta) no data' )
            print('\ts) review')
            print('\td) next')
            print('\tf) terminate')
            print('\tSelect a, s, d, f:')
            lts = input()
            if lts == "a":
                data_record['action'] = 'no data'
                data_record['issue'] = "no data availability or supplementary data mentioned in html or pdf versions or article"
                revised_list[int_idx] = data_record
                decide_action = True
            if lts == "s":
                data_record['action'] = 'review'
                print ('issue:',data_mentions[dm]['issue'])
                add_this = input()
                data_record['issue'] = add_this
                revised_list[int_idx] = data_record
                decide_action = True
            if lts == "d":
                decide_action = True
            elif lts == 'f':
                decide_action = True
                terminate = True
        if terminate:
            break

if len(revised_list) > 0:
    csvh.write_csv_data(revised_list, 'html_revised202111.csv')

In [ ]:
if len(revised_list) > 0:
    csvh.write_csv_data(revised_list, 'html_revised202111.csv')
revised_list

In [ ]:
# functions for ChemDataExtractor
# not used for mining data references (suplementary/raw) or to get pdf metadata
from chemdataextractor import Document

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

def cde_read_pdfs(a_file):
    pdf_f = open(a_file, 'rb')
    doc = Document.from_file(pdf_f)
    return doc

def find_doi(element_text):
    cr_re_01 = '10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.search(cr_re_01, element_text, re.IGNORECASE)
    if compare != None:
        return compare.group()
    return ""

def get_db_id(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "id", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_db_title(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "title", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_close_dois(str_name, db_name = "prev_search.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, doi, title, pdf_file"
    filter_str = "doi like '%"+str_name+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

Get the name of the current app db file:

In [ ]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = "db_files/app_db2.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
ukchapp_db



In [ ]:
# get names and links for references in data mentions
data_mentions, dm_fields = csvh.get_csv_data('pdf_mentions_filtered_02.csv', 'num')

for dm in data_mentions:
    print("https://doi.org/" + data_mentions[dm]['doi'])
    ref_name = data_mentions[dm]['ref_name']
    while ref_name == "":
        print('Please enter the name of data object:')
        ref_name = input()
    ref_link = data_mentions[dm]['ref_link']
    while ref_link == "":
        print('Please enter the data object link:')
        ref_link = input()
    data_mentions[dm]['ref_name'] = ref_name
    data_mentions[dm]['ref_link'] = ref_link


In [ ]:
len(data_records)

In [ ]:
data_mentions

In [ ]:
from inspect import getmembers, isfunction

In [ ]:
help(pdfminer.high_level)